<div align="center">
  
# ***Capacitated Lot-Sizing Problem with Setup Times (CLSP-ST)***
### *Group7*

</div>

### *Read file*

In [ ]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB, quicksum
import re
import matplotlib.pyplot as plt
import time


class Production_data:
    def __init__(self, number_of_periods, number_of_items, demand_forecast, production_cost, holding_cost, setup_cost, item_requirements , capacity):
        self.T=number_of_periods
        self.items=number_of_items
        self.demand_forecast=np.array(demand_forecast)
        self.production_cost=np.array(production_cost)
        self.holding_cost=np.array(holding_cost)
        self.setup_cost=np.array(setup_cost)
        self.item_requirements=np.array(item_requirements)
        self.capacity=np.array(capacity)

file_name = "CLSP+ST-instances Data-R.xlsx"
#file_name = "prova2.xlsx"

xls = pd.ExcelFile(file_name)  # Read the whole file

tables_keywords = ["Demand Forecast:", "Production Cost", "Holding Cost", "Setup Cost", "UnitsOfCapacity", "Capacity"]


In [ ]:

def read_data(xls, sheet_name):
    tables_dict = {}
    df = pd.read_excel(xls, sheet_name=sheet_name)
    
    # Define which column to check for each keyword
    columns_to_check = {
        "Demand Forecast:": 0,
        "Production Cost": 0,
        "Holding Cost": 0,
        "Setup Cost": 0,
        "UnitsOfCapacity": 1,
        "Capacity": 0
    }
    
    # Iterate through the keywords to find each table
    for keyword in tables_keywords:
        column_idx = columns_to_check.get(keyword, 0)
        
        # Check in the specified column for the keyword
        match = df[df.iloc[:, column_idx].astype(str).str.contains(keyword, na=False)]
        
        if not match.empty:
            table_start_row = match.index[0] + 1
            
            # Find the end of the current table (next keyword or empty rows)
            end_row = None
            for next_keyword in tables_keywords:
                if next_keyword != keyword:
                    next_column_idx = columns_to_check.get(next_keyword, 0)
                    next_match = df.loc[table_start_row:][df.loc[table_start_row:].iloc[:, next_column_idx].astype(str).str.contains(next_keyword, na=False)]
                    if not next_match.empty:
                        potential_end = next_match.index[0]
                        if end_row is None or potential_end < end_row:
                            end_row = potential_end
            
            # If no next keyword found, look for empty rows
            if end_row is None:
                for i in range(table_start_row, len(df)):
                    # Check if row is empty or contains only NaN values
                    if df.iloc[i].isna().all():
                        end_row = i
                        break
            
            # If still no end found, use the end of the dataframe
            if end_row is None:
                end_row = len(df)
            
            # Extract the table
            table_df = df.iloc[table_start_row:end_row]
            
            # Remove completely empty rows
            table_df = table_df.dropna(how='all')
            
            # Remove completely empty columns
            table_df = table_df.dropna(axis=1, how='all')
            
            # Remove any remaining NaN values by filling with 0
            table_df = table_df.fillna(0)
            
            # Convert to numpy array
            table = table_df.to_numpy()
            tables_dict[keyword] = table
    
    # Create an instance of Production_data class
    production_data = Production_data(
        tables_dict.get("Demand Forecast:", np.zeros((1,1))).shape[1]-1,
        tables_dict.get("Demand Forecast:", np.zeros((1,1))).shape[0]-1,  
        tables_dict.get("Demand Forecast:", np.zeros((1,1))),  
        tables_dict.get("Production Cost", np.zeros((1,1))),  
        tables_dict.get("Holding Cost", np.zeros((1,1))),  
        tables_dict.get("Setup Cost", np.zeros((1,1))),  
        tables_dict.get("UnitsOfCapacity", np.zeros((1,1))),  
        tables_dict.get("Capacity", np.zeros((1,1)))

    )
    
    production_data.capacity = np.vstack([
    np.zeros((1, production_data.capacity.shape[1]), dtype=production_data.capacity.dtype),production_data.capacity])

    production_data.item_requirements = np.vstack([
    np.zeros((1, production_data.item_requirements.shape[1]), dtype=production_data.item_requirements.dtype),production_data.item_requirements
])
    return production_data 


sheet_list=["Data-20-12 (1)", "Data-20-12 (2)", "Data-20-24 (1)", "Data-20-24 (2)", "Data-100-24 (1)", "Data-100-24 (2)", "Data-200-24"]


## **1.MILP vs LP Solutions**

### ***MILP***

In [ ]:
def solve_MILP(data, sheet_name, results_df):

    model=gp.Model("MiCLSP-ST")

    T=data.T
    O=data.items

    #decision variables, constraint 5 and 6 added here
    y=model.addVars(O+1, T+1, vtype=GRB.BINARY, name="y" ) #produce of not for this item in this period
    s=model.addVars(O+1, T+1, vtype=GRB.CONTINUOUS, lb=0, name="s") #amount of inv of item i in period t
    x=model.addVars(O+1, T+1, vtype=GRB.CONTINUOUS, lb=0, name="x") #amount produced of item i in period t

    model.setObjective(
        quicksum(quicksum( data.setup_cost[i, t] * y[i, t] for i in range(1, O+1) ) for t in range(1, T+1))+
        quicksum(quicksum( data.production_cost[i, t] * x[i, t] for i in range(1, O+1) ) for t in range(1, T+1))+
        quicksum(quicksum( data.holding_cost[i, t] * s[i, t] for i in range(1, O+1) ) for t in range(1, T+1)),
        GRB.MINIMIZE
    )  


    #constraint 2
    for t in range(1, T+1):
        for i in range(1, O+1):
            model.addConstr(s[i, t-1]+x[i, t]-s[i, t]==data.demand_forecast[i, t])

    

    #constraint 3, this definitely works
    for t in range(1, T+1):
        print(data.capacity[t,1])
        model.addConstr( quicksum( x[i, t]*data.item_requirements[i, 1] + data.item_requirements[i,2] * y[i,t] for i in range(1, O+1))<= data.capacity[t, 1])
        
    #constraint 4, this defintely ok
    for t in range(1, T+1):
        for i in range(1, O+1):
            model.addConstr(x[i, t]-(quicksum(data.demand_forecast[i, q] for q in range(t, T+1))) * y[i, t]<=0)

    #constraint 7, no initial inventory for all items, this definitely works
    for i in range(1, O+1):
        model.addConstr( s[i, 0] == 0, name="no init inv") #lloks like this constrint is ok
    
    #solve the model
    #model.setParam.MIPGap = 0.05
    model.setParam('TimeLimit', 20*60)
    model.optimize()

    if model.Status == GRB.OPTIMAL:
        print("\noptimal solution found:")
        print(model.ObjVal)

        # Create a list to store all periods' results for this sheet
        period_results = []
        
        # Collect results for each period
        for t in range(1, T+1):
            period_results.append({
                "Sheet": sheet_name,
                "Period": t,
                "Inventory of item 1": s[1, t].X, 
                "Produced for item 1": x[1, t].X
            })
        
        # Add all periods at once to the results DataFrame
        sheet_results = pd.DataFrame(period_results)
        results_df = pd.concat([results_df, sheet_results], ignore_index=True)
    
    return results_df

In [ ]:
# Initialize results DataFrame with all needed columns
results_df = pd.DataFrame(columns=["Sheet", "Period", "Inventory of item 1", "Produced for item 1"])

for sheet_name in sheet_list:
    data = read_data(xls, sheet_name=sheet_name)
    results_df = solve_MILP(data, sheet_name, results_df)

results_df.to_csv("results_MILP.csv", index=False)

### ***LP-relaxation vs MILP Results***

In [ ]:
def solve_model(data, sheet_name, results_df, relaxation=False):
    model = gp.Model("MiCLSP-ST")
    T = data.T
    O = data.items

    # Set silent mode
    model.setParam("OutputFlag", 0)

    # Variables
    vtype = GRB.CONTINUOUS if relaxation else GRB.BINARY
    y = model.addVars(O + 1, T + 1, vtype=vtype, name="y")
    s = model.addVars(O + 1, T + 1, vtype=GRB.CONTINUOUS, lb=0, name="s")
    x = model.addVars(O + 1, T + 1, vtype=GRB.CONTINUOUS, lb=0, name="x")

    # Objective
    model.setObjective(
        quicksum(data.setup_cost[i, t] * y[i, t] for i in range(1, O + 1) for t in range(1, T + 1)) +
        quicksum(data.production_cost[i, t] * x[i, t] for i in range(1, O + 1) for t in range(1, T + 1)) +
        quicksum(data.holding_cost[i, t] * s[i, t] for i in range(1, O + 1) for t in range(1, T + 1)),
        GRB.MINIMIZE
    )

    # Constraints
    for t in range(1, T + 1):
        for i in range(1, O + 1):
            model.addConstr(s[i, t - 1] + x[i, t] - s[i, t] == data.demand_forecast[i, t])

    for t in range(1, T + 1):
        model.addConstr(
            quicksum(x[i, t] * data.item_requirements[i, 1] + data.item_requirements[i, 2] * y[i, t]
                     for i in range(1, O + 1)) <= data.capacity[t, 1]
        )

    for t in range(1, T + 1):
        for i in range(1, O + 1):
            model.addConstr(x[i, t] <= quicksum(data.demand_forecast[i, q] for q in range(t, T + 1)) * y[i, t])

    for i in range(1, O + 1):
        model.addConstr(s[i, 0] == 0)

    # Solve and measure time
    start_time = time.time()
    model.setParam('TimeLimit', 20*60) 
    model.optimize()
    solve_time = time.time() - start_time

    # Handle solution
    if model.Status in [GRB.OPTIMAL, GRB.TIME_LIMIT] and model.SolCount > 0:
        label = "LP Relaxation" if relaxation else "MILP"
        objective = model.ObjVal

        for t in range(1, T + 1):
            new_row = pd.DataFrame([{
                "Sheet": sheet_name,
                "Model Type": label,
                "Period": t,
                "Inventory of item 1": s[1, t].X,
                "Produced for item 1": x[1, t].X,
                "Objective Value": objective,
                "Solve Time (s)": solve_time
            }])
            results_df = pd.concat([results_df, new_row], ignore_index=True)

        return results_df, objective, solve_time, "Optimal"

    # If no feasible solution found
    print(f"No feasible solution for {sheet_name} ({'Relaxed' if relaxation else 'MILP'}). Status: {model.Status}")
    return results_df, float('nan'), float('nan'), "infeasible"


In [ ]:
summary_df = pd.DataFrame(columns=[
    "Sheet", "ZLP", "ZIP", "Time_LP", "Time_IP", "Status_LP", "Status_IP"
])
results = pd.DataFrame(columns=[
    "Sheet", "Model Type", "Period", "Inventory of item 1",
    "Produced for item 1", "Objective Value", "Solve Time (s)"
])

for sheet in sheet_list:
    print(f"Processing {sheet}")
    data = read_data(xls, sheet)

    # Solve MILP
    results, ZIP, Time_IP, status_IP = solve_model(data, sheet, results, relaxation=False)

    # Solve LP Relaxation
    results, ZLP, Time_LP, status_LP = solve_model(data, sheet, results, relaxation=True)

    summary_df = pd.concat([summary_df, pd.DataFrame([{
        "Sheet": sheet,
        "ZLP": ZLP,
        "ZIP": ZIP,
        "Time_LP": Time_LP,
        "Time_IP": Time_IP,
        "Status_LP": status_LP,
        "Status_IP": status_IP
    }])], ignore_index=True)

# Save results
summary_df.to_csv("MIP_vs_LP-relax_results.csv", index=False)
print("Saved detailed and summary results.")

## **2.Plant Location Reformulation**

As for the single-item lot-sizing problem, an alternative formulation for the current problem, also 
called plant location reformulation (PLRFIP), is obtained by replacing in the model (MiCLSP-STIP) the 
production variables 𝑥𝑖𝑡 by the variables 𝑤𝑠𝑡 𝑖 given by:

Where 𝑤𝑠𝑡 𝑖 can be interpreted as the portion of demand of item 𝑖 ∈ 𝑃 in period 𝑡 ∈ 𝐻 fulfilled by 
production in period 𝑠 ∈ 𝐻,𝑠 ≤ 𝑡. Write down this reformulation and check the validity of your 
proposed model. 

In [ ]:
def solve_PLRF(data, sheet_name, results_df):

    model=gp.Model("PLRF")


    #decisions variables

    T=data.T
    O=data.items

    w=model.addVars(O+1, T+1, T+1, vtype=GRB.CONTINUOUS, lb=0, ub=1, name = "w" )#fraction of demand of item i produced in period s to satisfy period t
    y=model.addVars(O+1, T+1, vtype=GRB.BINARY, name="y") #produce or not for item i in period i

    model.setObjective(
        #setup cost
        quicksum(quicksum( data.setup_cost[i, t] * y[i, t] for i in range(1, O+1) ) for t in range(1, T+1))+
        #production cost, j is actual period and t in future period
        quicksum(quicksum(quicksum(data.production_cost[i, s] * data.demand_forecast[i,t] * w[i, s, t] 
                          for t in range(s, T+1)) # Changed range
                 for s in range(1, T+1))
        for i in range(1, O+1))+
        #holding cost, i per item
        quicksum( quicksum(quicksum
                           (sum(data.holding_cost[ i, t] for t in range(s, j)) * data.demand_forecast[i, j] * w[i, s, j] for j in range(s+1, T+1))
                           for s in range(1, T+1)) for i in range(1, O+1)),
        GRB.MINIMIZE
    )
    
    

    
    # Each demand must be fully satisfied (like each customer must be served) for each item
    for i in range(1, O+1):
        for t in range(1, T+1):
            model.addConstr( quicksum(w[i, s, t] for s in range(1, t+1)) == 1, name= f"demand_{i}{t}")
            
    #cannot produce any fraction of demand in period s is the variable y is 0, for item i
    for i in range(1, O+1):
        for t in range(1, T+1):
            for s in range(1, t+1):
                model.addConstr( w[i, s, t] <= y[i, s] )
        
    #still capacity constraints fossure
    #constraint 3

    for s in range(1, T+1):
        model.addConstr(
            quicksum(
                # Production time: sum over all demands being produced in period s
                sum(w[i, s, t] * data.demand_forecast[i, t] for t in range(s, T+1)) * data.item_requirements[i, 1] +
                # Setup time
                data.item_requirements[i, 2] * y[i, s]
                for i in range(1, O+1)
            ) <= data.capacity[s, 1],
            name=f"capacity_{s}"
        )
        
    
    #constraint 4
    for t in range(1, T+1):
        for i in range(1, O+1):
            model.addConstr( sum( w[i, t, s] for s in range(t, T+1) )-(quicksum(data.demand_forecast[i, q] for q in range(t, T+1))) * y[i, t]<=0)

    
    #solve
    model.setParam('TimeLimit', 20*60)
    model.optimize()

    if model.status == GRB.OPTIMAL:

        print("\noptimal solution found:")

        # Create a list to store all periods' results for this sheet
        period_results = []
        
        # Collect results for each period
        for t in range(1, T+1):
            period_results.append({
                "Sheet": sheet_name,
                "Period": t,
                "Setup Decision": y[1, t].X,
                "Production Quantity": sum(w[2, t, j].X * data.demand_forecast[2, j] for j in range(t, T+1)),
                "Capacity Usage": sum(sum(w[i, t, j].X * data.demand_forecast[i, j] * data.item_requirements[i, 1] + data.item_requirements[i, 2] * y[i,t].X  
                                        for j in range(t, T+1)) 
                                    for i in range(1, O+1)) / data.capacity[t, 1],
                "Total Cost": model.objVal
            })
        
        sheet_results = pd.DataFrame(period_results)
        results_df = pd.concat([ results_df, sheet_results ], ignore_index=False)


    return results_df


In [ ]:

# Initialize results DataFrame with all needed columns
results_df = pd.DataFrame(columns=["Sheet", "Period", "Produced in the period for item 1"])

for sheet_name in sheet_list:
    data = read_data(xls, sheet_name=sheet_name)
    results_df = solve_PLRF(data, sheet_name, results_df)

results_df.to_csv("results_PLRF.csv", index=False)

### **LP-relaxation (PLRFLP) of the (PLRFIP)**

In [ ]:
def solve_PLRF_models(data, sheet_name, results_df):
    T = data.T
    O = data.items

    def build_and_solve(relax=False):
        model = gp.Model("PLRF_LP" if relax else "PLRF_IP")
        model.Params.OutputFlag = 0      # Suppress output
        model.Params.TimeLimit = 20*60     

        # Variables
        w = model.addVars(O+1, T+1, T+1, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="w")
        y = model.addVars(O+1, T+1, vtype=GRB.CONTINUOUS if relax else GRB.BINARY, name="y")

        # Objective
        model.setObjective(
            quicksum(quicksum(data.setup_cost[i, t] * y[i, t] for i in range(1, O+1)) for t in range(1, T+1)) +
            quicksum(quicksum(quicksum(
                data.production_cost[i, s] * data.demand_forecast[i, t] * w[i, s, t]
                for t in range(s, T+1)) for s in range(1, T+1)) for i in range(1, O+1)) +
            quicksum(quicksum(quicksum(
                sum(data.holding_cost[i, t] for t in range(s, j)) * data.demand_forecast[i, j] * w[i, s, j]
                for j in range(s+1, T+1)) for s in range(1, T+1)) for i in range(1, O+1)),
            GRB.MINIMIZE
        )

        # Constraints
        for i in range(1, O+1):
            for t in range(1, T+1):
                model.addConstr(quicksum(w[i, s, t] for s in range(1, t+1)) == 1)

        for i in range(1, O+1):
            for t in range(1, T+1):
                for s in range(1, t+1):
                    model.addConstr(w[i, s, t] <= y[i, s])

        for s in range(1, T+1):
            model.addConstr(
                quicksum(
                    sum(w[i, s, t] * data.demand_forecast[i, t] for t in range(s, T+1)) * data.item_requirements[i, 1] +
                    data.item_requirements[i, 2] * y[i, s]
                    for i in range(1, O+1)
                ) <= data.capacity[s, 1]
            )

        for t in range(1, T+1):
            for i in range(1, O+1):
                model.addConstr(
                    sum(w[i, t, s] for s in range(t, T+1)) - 
                    quicksum(data.demand_forecast[i, q] for q in range(t, T+1)) * y[i, t] <= 0
                )

        model.optimize()
        runtime = model.Runtime

        if model.Status in [GRB.OPTIMAL, GRB.TIME_LIMIT] and model.SolCount > 0:
            obj_val = model.ObjVal
        else:
            obj_val = None

        return obj_val, runtime

    # Solve both models
    rfz_ip, time_ip = build_and_solve(relax=False)
    rfz_lp, time_lp = build_and_solve(relax=True)

    # Save results
    results_df = pd.concat([results_df, pd.DataFrame([{
        "Sheet": sheet_name,
        "RFZ_IP": rfz_ip,
        "Time_IP (s)": time_ip,
        "RFZ_LP": rfz_lp,
        "Time_LP (s)": time_lp
    }])], ignore_index=True)

    return results_df


In [ ]:
summary_df = pd.DataFrame(columns=["Sheet", "RFZLP", "RFZIP", "Time_LP", "Time_IP"])
results_df = pd.DataFrame(columns=[
    "Sheet", "RFZ_IP", "Time_IP (s)", "RFZ_LP", "Time_LP (s)"
])

for sheet in sheet_list:
    print(f"Processing {sheet}")
    data = read_data(xls, sheet)

    # Append both LP and IP results to results_df
    results_df = solve_PLRF_models(data, sheet, results_df)

# Save results
results_df.to_csv("results_PLRF_relax.csv", index=False)

 ## **3.Lagrangian Relaxation LR PLRF**

In [2]:
class Production_data:
    def __init__(self, number_of_periods, number_of_items, demand_forecast, production_cost, holding_cost, setup_cost, item_requirements , capacity):
        self.T=number_of_periods
        self.items=number_of_items
        self.demand_forecast=np.array(demand_forecast)
        self.production_cost=np.array(production_cost)
        self.holding_cost=np.array(holding_cost)
        self.setup_cost=np.array(setup_cost)
        self.item_requirements=np.array(item_requirements)
        self.capacity=np.array(capacity)

file_name = "CLSP+ST-instances Data-R.xlsx"
#file_name = "prova2.xlsx"

xls = pd.ExcelFile(file_name)  # Read the whole file

tables_keywords = ["Demand Forecast:", "Production Cost", "Holding Cost", "Setup Cost", "UnitsOfCapacity", "Capacity"]

In [3]:
def read_data(xls, sheet_name):
    tables_dict = {}
    df = pd.read_excel(xls, sheet_name=sheet_name)
    
    # Define which column to check for each keyword
    columns_to_check = {
        "Demand Forecast:": 0,
        "Production Cost": 0,
        "Holding Cost": 0,
        "Setup Cost": 0,
        "UnitsOfCapacity": 1,  # Check the second column for this keyword (it also contains setup time(i,2))
        "Capacity": 0
    }
    
    # Iterate through the keywords to find each table
    for keyword in tables_keywords:
        column_idx = columns_to_check.get(keyword, 0)
        
        # Check in the specified column for the keyword
        match = df[df.iloc[:, column_idx].astype(str).str.contains(keyword, na=False)]
        
        if not match.empty:
            table_start_row = match.index[0] + 1
            
            # Find the end of the current table (next keyword or empty rows)
            end_row = None
            for next_keyword in tables_keywords:
                if next_keyword != keyword:
                    next_column_idx = columns_to_check.get(next_keyword, 0)
                    next_match = df.loc[table_start_row:][df.loc[table_start_row:].iloc[:, next_column_idx].astype(str).str.contains(next_keyword, na=False)]
                    if not next_match.empty:
                        potential_end = next_match.index[0]
                        if end_row is None or potential_end < end_row:
                            end_row = potential_end
            
            # If no next keyword found, look for empty rows
            if end_row is None:
                for i in range(table_start_row, len(df)):
                    # Check if row is empty or contains only NaN values
                    if df.iloc[i].isna().all():
                        end_row = i
                        break
            
            # If still no end found, use the end of the dataframe
            if end_row is None:
                end_row = len(df)
            
            # Extract the table
            table_df = df.iloc[table_start_row:end_row]
            
            # Remove completely empty rows
            table_df = table_df.dropna(how='all')
            
            # Remove completely empty columns
            table_df = table_df.dropna(axis=1, how='all')
            
            # Remove any remaining NaN values by filling with 0
            table_df = table_df.fillna(0)
            
            # Convert to numpy array
            table = table_df.to_numpy()
            tables_dict[keyword] = table
    
    # Create an instance of Production_data class
    production_data = Production_data(
        tables_dict.get("Demand Forecast:", np.zeros((1,1))).shape[1]-1,
        tables_dict.get("Demand Forecast:", np.zeros((1,1))).shape[0]-1,  
        tables_dict.get("Demand Forecast:", np.zeros((1,1))),  
        tables_dict.get("Production Cost", np.zeros((1,1))),  
        tables_dict.get("Holding Cost", np.zeros((1,1))),  
        tables_dict.get("Setup Cost", np.zeros((1,1))),  
        tables_dict.get("UnitsOfCapacity", np.zeros((1,1))),  
        tables_dict.get("Capacity", np.zeros((1,1)))

    )
    
    production_data.capacity = np.vstack([
    np.zeros((1, production_data.capacity.shape[1]), dtype=production_data.capacity.dtype),production_data.capacity])

    production_data.item_requirements = np.vstack([
    np.zeros((1, production_data.item_requirements.shape[1]), dtype=production_data.item_requirements.dtype),production_data.item_requirements
])
    return production_data 

In [4]:
def solve_PLRF(data, sheet_name, results_df):

    model=gp.Model("PLRF")


    #decisions variables

    T=data.T
    O=data.items

    w=model.addVars(O+1, T+1, T+1, vtype=GRB.CONTINUOUS, lb=0, ub=1, name = "w" )#fraction of demand of item i produced in period s to satisfy period t
    y=model.addVars(O+1, T+1, vtype=GRB.BINARY, name="y") #produce or not for item i in period i

    model.setObjective(
        #setup cost
        quicksum(quicksum( data.setup_cost[i, t] * y[i, t] for i in range(1, O+1) ) for t in range(1, T+1))+
        #production cost, j is actual period and t in future period
        quicksum(quicksum(quicksum(data.production_cost[i, s] * data.demand_forecast[i,t] * w[i, s, t] 
                          for t in range(s, T+1)) # Changed range
                 for s in range(1, T+1))
        for i in range(1, O+1))+
        #holding cost, i per item
        quicksum( quicksum(quicksum
                           (sum(data.holding_cost[ i, t] for t in range(s, j)) * data.demand_forecast[i, j] * w[i, s, j] for j in range(s+1, T+1))
                           for s in range(1, T+1)) for i in range(1, O+1)),
        GRB.MINIMIZE
    )
    
    

    
    # Each demand must be fully satisfied (like each customer must be served) for each item
    for i in range(1, O+1):
        for t in range(1, T+1):
            model.addConstr( quicksum(w[i, s, t] for s in range(1, t+1)) == 1, name= f"demand_{i}{t}")
            
    #cannot produce any fraction of demand in period s is the variable y is 0, for item i
    for i in range(1, O+1):
        for t in range(1, T+1):
            for s in range(1, t+1):
                model.addConstr( w[i, s, t] <= y[i, s] )
        
    #still capacity constraints fossure
    #constraint 3

    for s in range(1, T+1):
        model.addConstr(
            quicksum(
                # Production time: sum over all demands being produced in period s
                sum(w[i, s, t] * data.demand_forecast[i, t] for t in range(s, T+1)) * data.item_requirements[i, 1] +
                # Setup time
                data.item_requirements[i, 2] * y[i, s]
                for i in range(1, O+1)
            ) <= data.capacity[s, 1],
            name=f"capacity_{s}"
        )
        
    
    #constraint 4
    for t in range(1, T+1):
        for i in range(1, O+1):
            model.addConstr( sum( w[i, t, s] for s in range(t, T+1) )-(quicksum(data.demand_forecast[i, q] for q in range(t, T+1))) * y[i, t]<=0)

    
    #solve
    model.setParam('TimeLimit', 20*60)
    model.optimize()

    if model.status == GRB.OPTIMAL:

        print("\noptimal solution found:")

        # Create a list to store all periods' results for this sheet
        period_results = []
        
        # Collect results for each period
        for t in range(1, T+1):
            period_results.append({
                "Sheet": sheet_name,
                "Period": t,
                "Setup Decision": y[1, t].X,
                "Production Quantity": sum(w[2, t, j].X * data.demand_forecast[2, j] for j in range(t, T+1)),
                "Capacity Usage": sum(sum(w[i, t, j].X * data.demand_forecast[i, j] * data.item_requirements[i, 1] + data.item_requirements[i, 2] * y[i,t].X  
                                        for j in range(t, T+1)) 
                                    for i in range(1, O+1)) / data.capacity[t, 1],
                "Total Cost": model.objVal
            })
        
        sheet_results = pd.DataFrame(period_results)
        results_df = pd.concat([ results_df, sheet_results ], ignore_index=False)


    return results_df, model.ObjVal

In [ ]:
results_df = pd.DataFrame(columns=["Sheet", "Period", "Inventory of item 1", "Produced for item 1"])
sheet_list=["Data-20-12 (1)", "Data-20-12 (2)", "Data-20-24 (1)", "Data-20-24 (2)" , "Data-100-24 (1)", "Data-100-24 (2)",
            "Data-200-24" ]

ObjVals = []

for sheet_name in sheet_list:
    break
    print(sheet_name)
    data = read_data(xls, sheet_name=sheet_name)
    
    start = time.time()
    results_df, Z = solve_PLRF(data, sheet_name, results_df)
    end = time.time()
    ObjVals.append((Z, end-start))



In [ ]:
def Lagrangian_Relaxation_PLRF(data, optimal_value):
    model = gp.Model("PLRF_LR")

    T = data.T
    O = data.items

    # Initialize Lagrange multipliers
    mu = np.ones(T + 1)  # +1 because periods start from 1
    dual_bounds = []
    best_dual_bound = float('-inf')
    
    iteration = 0
    max_iterations = 1000
    tolerance = 0.05
    
    start = time.time()
    end = time.time()
    while iteration <= max_iterations and end-start <= 10*60:
        iteration += 1
        print(f'Iteration: {iteration}')
        
        # Decision variables
        w = model.addVars(O+1, T+1, T+1, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="w")
        y = model.addVars(O+1, T+1, vtype=GRB.BINARY, name="y")
        
        # Objective function with relaxed capacity constraints
        model.setObjective(
            # Setup cost
            quicksum(quicksum(data.setup_cost[i, t] * y[i, t] for i in range(1, O+1)) for t in range(1, T+1)) +
            # Production cost
            quicksum(quicksum(quicksum(data.production_cost[i, s] * data.demand_forecast[i,t] * w[i, s, t] 
                              for t in range(s, T+1))
                     for s in range(1, T+1))
            for i in range(1, O+1)) +
            # Holding cost
            quicksum(quicksum(quicksum(
                       sum(data.holding_cost[i, t] for t in range(s, j)) * data.demand_forecast[i, j] * w[i, s, j] 
                       for j in range(s+1, T+1))
                   for s in range(1, T+1)) for i in range(1, O+1)) +
            # Lagrangian term for relaxed capacity constraints
            quicksum(mu[s] * (
                quicksum(
                    sum(w[i, s, t] * data.demand_forecast[i, t] for t in range(s, T+1)) * data.item_requirements[i, 1] +
                    data.item_requirements[i, 2] * y[i, s]
                    for i in range(1, O+1)
                ) - data.capacity[s, 1]
            ) for s in range(1, T+1)),
            GRB.MINIMIZE
        )
        
        # Original constraints (excluding the relaxed capacity constraints)
        # Each demand must be fully satisfied
        for i in range(1, O+1):
            for t in range(1, T+1):
                model.addConstr(quicksum(w[i, s, t] for s in range(1, t+1)) == 1, name=f"demand_{i}{t}")
        
        # Production fraction constraint
        for i in range(1, O+1):
            for t in range(1, T+1):
                for s in range(1, t+1):
                    model.addConstr(w[i, s, t] <= y[i, s])
        
        # Additional constraint from original formulation
        for t in range(1, T+1):
            for i in range(1, O+1):
                model.addConstr(sum(w[i, t, s] for s in range(t, T+1)) - 
                               (quicksum(data.demand_forecast[i, q] for q in range(t, T+1))) * y[i, t] <= 0)
        
        model.optimize()
        end = time.time()

        if model.status == GRB.OPTIMAL:
            dual_obj = model.ObjVal
            dual_bounds.append(dual_obj)
            best_dual_bound = max(best_dual_bound, dual_obj)
            
            # Calculate subgradients for capacity constraints
            subgradient = np.zeros(T + 1)
            for s in range(1, T+1):
                subgradient[s] = sum(
                    sum(w[i, s, t].X * data.demand_forecast[i, t] for t in range(s, T+1)) * data.item_requirements[i, 1] +
                    data.item_requirements[i, 2] * y[i, s].X
                    for i in range(1, O+1)
                ) - data.capacity[s, 1]
            
            # Update Lagrange multipliers
            step_size = 1.0 / iteration  # Decreasing step size
            # mu = np.maximum(0, mu + step_size * subgradient)
            mu = np.maximum(0, mu + subgradient/np.max(np.abs(subgradient)*10))
            
            # Check for convergence
            if iteration > 25:
                recent_avg = np.mean(dual_bounds[-25:])
                if np.abs(recent_avg - best_dual_bound) / (best_dual_bound + 1e-6) < tolerance:
                    break
        else:
            print("Model not optimal in iteration", iteration)
            break
    
    gap = (optimal_value - best_dual_bound) / optimal_value if optimal_value != 0 else 0

    plt.figure(figsize=(10, 6))
    plt.plot(dual_bounds)
    plt.axhline(best_dual_bound, linestyle='--', color='red', linewidth=0.5, 
                label=f'Best Dual Bound: {best_dual_bound:.2f}')
    plt.axhline(optimal_value, linestyle='--', color='b', linewidth=0.5, 
                label=f'Optimal value: {optimal_value:.2f}')
    plt.title(f'Dual Bounds, Gap: {round(gap, 4)*100} %')
    plt.legend()
    plt.show()

    return best_dual_bound

In [ ]:
results_LR = []
sheet_list=["Data-20-12 (1)", "Data-20-12 (2)", "Data-20-24 (1)", "Data-20-24 (2)" , "Data-100-24 (1)", "Data-100-24 (2)",
            "Data-200-24" ]

optimal_values = {'Data-20-12 (1)': 89469.0,
                  'Data-20-12 (2)': 14330786.370279722,
                  'Data-20-24 (1)': 5227122.577272725,
                  'Data-20-24 (2)': 6711666.365537237,
                  'Data-100-24 (1)': 37944696.08376624,
                  'Data-100-24 (2)': 37944696.08376624,
                  'Data-200-24': 75798079.88268259}

In [ ]:
sheet_name = sheet_list[0]
print(sheet_name)
data = read_data(xls, sheet_name=sheet_name)

start = time.time()
Z = Lagrangian_Relaxation_PLRF(data, optimal_values[sheet_name])
end = time.time()

results_LR.append((Z, end-start))

## **4.DantzigWolfe**

In [5]:
def get_data(sheet_name):
    file_path = "CLSP+ST-instances Data-R.xlsx"  
    numbers = re.findall(r'\d+', sheet_name)
    number_items = int(numbers[0])
    number_capacity = int(numbers[1])
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)

    # Identify matrix boundaries manually or by searching keywords
    demand_start = df[df[0] == "Demand Forecast:"].index[0] + 1
    production_start = df[df[0] == "Production Cost"].index[0] + 1
    holding_start = df[df[0] == "Holding Cost"].index[0] + 1
    setup_start = df[df[0] == "Setup Cost"].index[0] + 1
    capacity_start = df[df[0] == "Capacity"].index[0] + 1

    # Extract matrices and remove first row and first column
    demand_forecast = df.iloc[demand_start+1:production_start-2, 1:].reset_index(drop=True)
    production_cost = df.iloc[production_start+1:holding_start-2, 1:].reset_index(drop=True)
    holding_cost = df.iloc[holding_start+1:setup_start-2, 1:].reset_index(drop=True)
    setup_cost = df.iloc[setup_start+1:setup_start+number_items+1, 1:].reset_index(drop=True)
    capacity = df.iloc[capacity_start:capacity_start+number_capacity+1, 1:2].reset_index(drop=True)

    UnitsOfCapacity_SupTime = df.iloc[setup_start+number_items+2:capacity_start-2, 1:3].reset_index(drop=True)
    UnitsOfCapacity_SupTime.columns = UnitsOfCapacity_SupTime.iloc[0]
    UnitsOfCapacity_SupTime = UnitsOfCapacity_SupTime[1:].reset_index(drop=True)
    UnitsOfCapacity = UnitsOfCapacity_SupTime['UnitsOfCapacity']
    SetupTime = UnitsOfCapacity_SupTime['SetupTime']

    # Convert to numeric values
    demand_forecast = demand_forecast.apply(pd.to_numeric, errors='coerce')
    production_cost = production_cost.apply(pd.to_numeric, errors='coerce')
    holding_cost = holding_cost.apply(pd.to_numeric, errors='coerce')
    setup_cost = setup_cost.apply(pd.to_numeric, errors='coerce')

    data = {}
    data['Demand Forecast'] = demand_forecast
    data['Production Cost'] = production_cost
    data['Holding Cost'] = holding_cost
    data['Setup Cost'] = setup_cost
    data['UnitsOfCapacity'] = UnitsOfCapacity
    data['SetupTime'] = SetupTime
    data['Capacity'] = capacity
    return data

In [6]:
# data = get_data('Data-20-12 (2)')
# data.keys()

def defineData(sheet):

    #print('Defining Data -----')
    data = get_data(sheet)
    
    N, T = data['Demand Forecast'].shape
    
    #define data + add padding for dummy periods and products
    demand = np.pad(np.array(data['Demand Forecast']), ((1,0), (1,0))) #All demand demand[producttype, time]
    c = np.pad(np.array(data['Production Cost']), ((1,0), (1,0))) #Production Cost [producttype, time]
    h = np.pad(np.array(data['Holding Cost']), ((1,0), (1,0))) #holding Cost ||
    f = np.pad(np.array(data['Setup Cost']), ((1,0), (1,0))) #Setup cost ||
    r = np.pad(np.array(data['UnitsOfCapacity']), (1, 0)) #Capacity units (only in N)
    kappa = np.pad(np.array(data['Capacity']).flatten(), (1,0)) #Total available Capacity (only in T)
    tau = np.pad(np.array(data['SetupTime']), (1,0)) #Setup Time (Only in N)
    
    #Initial Inventory Cost to force feasibility for the RMP
    initialCost = 1e4

    return N, T, demand, c, h, f, r, kappa, tau, initialCost


In [7]:
#Trivial plans
def initial_columns():
    columns = {i: [] for i in range(N+1)}

    for i in range(N+1):
        plan = {
            'x': [0] * (T+1),
            'y': [0] * (T+1),
            's': [sum(demand[i, :])] + ([0] * T),
            'cost': sum(demand[i, :])*initialCost,
            'cap': [0] * (T+1)       
        }
        columns[i].append(plan)
    return columns

In [8]:
# ------------------------------
# STEP 3: Restricted Master Problem (RMP)
# ------------------------------
def solve_rmp(columns):
    
    #print("--------Solving RMP----------")
    model = gp.Model("RMP")
    model.setParam('OutputFlag', 0)

    #Define the weights
    z = {}
    for i in range(1, N+1): #iterate over each product and plan -> These are the weights without dummy 
        for k, _ in enumerate(columns[i]):
            z[i, k] = model.addVar(vtype=GRB.CONTINUOUS, name=f"z_{i}_{k}")

    model.update()
    
    #RMP Objective
    model.setObjective(gp.quicksum(plan['cost'] * z[i, k] for i in range(1, N+1) for k, plan in enumerate(columns[i])), GRB.MINIMIZE)

   
    #Capacity respected in each period
    for t in range(1, T+1):
        constr = model.addConstr(
            gp.quicksum(plan['cap'][t]*z[i, k] for i in range(1, N+1) for k, plan in enumerate(columns[i])) <= kappa[t],
            name=f"cap_{t}"
        )
    
    for i in range(1, N+1): #Exactly one schedule per product
        model.addConstr(gp.quicksum(z[i, k] for k in range(len(columns[i]))) == 1, name=f"item_{i}")

    
    model.update()
    model.optimize()

    duals = model.getAttr(GRB.Attr.Pi)
    #print(len(duals), duals)
    #for c in model.getConstrs():
        #print(f"{c.ConstrName} dual: {c.Pi}")

    model.update()

    z_values = {(i, k): z[i, k].X for i in range(1, N+1) for k, _ in enumerate(columns[i])}

    return model.ObjVal, duals, z_values

In [9]:
# ------------------------------
# STEP 4: Subproblem (Pricing)
# ------------------------------
def solve_subproblem(i, T, demand_i, f_i, c_i, h_i, r_i, tau_i, duals):

    #print(f"--------Solving Subproblem product {i}----------")
    
    model = gp.Model(f"Subproblem_{i}")
    model.setParam('OutputFlag', 0)

    x = model.addVars(T+1, vtype=GRB.INTEGER, name="x") #With dummy period
    s = model.addVars(T+1, vtype=GRB.INTEGER, name="s")
    y = model.addVars(T+1, vtype=GRB.BINARY, name="y")

    dual_capacity = np.pad(duals[:T], (1,0)) #dummy period
    dual_convexity = np.pad(duals[T:], (1,0)) #dummy product
    #print(dual_capacity)
    #print(dual_convexity)


    #Minimize the reduced cost, if it is negative, it can improve the reduced master problem.
    #Cost of plan - D * Pi)
    model.setObjective(
        s[0] * initialCost + gp.quicksum( f_i[t]*y[t] + c_i[t]*x[t] + h_i[t]*s[t] for t in range(1, T+1)) #Cost of Plan
        - gp.quicksum(dual_capacity[t]*(r_i*x[t] + tau_i*y[t]) for t in range(1, T+1)) #reduced costs capacity
    , GRB.MINIMIZE)


    # Constraints
    model.addConstr(s[0] == 0, name="no_inventory0") #-> To enforce feasibility
    model.addConstr(s[T] == 0, name="no_inventoryT")
    
    for t in range(1, T+1):
        model.addConstr(s[t-1] + x[t] == demand_i[t] + s[t], name=f"demand_satisfied_{t}_{i}") #demand satisfied 
        model.addConstr(x[t] <= gp.quicksum(demand_i[m] for m in range(t, T+1)) * y[t], name=f"setup_constraint_{t}")

    model.optimize()
    model.update()
    
    if model.objVal - dual_convexity[i]  < -1e-6: 
        #print("--Better Schedule Found--")
        plan = {
            'x': [x[t].X for t in range(T+1)],
            'y': [y[t].X for t in range(T+1)],
            's': [s[t].X for t in range(T+1)],
            'cost': s[0].X * initialCost + sum(f_i[t]*y[t].X + c_i[t]*x[t].X + h_i[t]*s[t].X for t in range(1, T+1)),
            'cap': [r_i*x[t].X + tau_i*y[t].X for t in range(T+1)]
        }
        #print(plan)
        #print('Reduced Cost:', sum(dual_capacity[t]*(r_i*x[t].X + tau_i*y[t].X) for t in range(1, T+1)))
        return plan
    return None

In [10]:
#step 5 column generation

def column_generation():
    columns = initial_columns()
    converged = False
    iteration = 0
    start_time = time.time()

    while not converged:
        iteration += 1
        #print(f"\n=== Iteration {iteration} ===")

        final_obj, duals, z = solve_rmp(columns)
        dual_capacity = duals[:T]
        dual_convexity = duals[T:]

        #print(f"RMP Objective: {final_obj}")
        #print(f"Dual capacity: {dual_capacity}")
        #print(f"Dual convexity: {dual_convexity}")
        
        #print(z)

        new_columns = 0

        for i in range(1, N+1):
            new_plan = solve_subproblem(i, T, demand[i], f[i], c[i], h[i], r[i], tau[i], duals)
            if new_plan:
                columns[i].append(new_plan)
                new_columns += 1

        #print(f"Current Objective Value = {final_obj}")

        # Always show current plans per product
        # print(f"\n--- Current plans per product (only x's) ---")
        # for i in range(1, N+1):
        #     print(f"Product {i}:")
        #     for idx, plan in enumerate(columns[i]):
        #         print(f"  Column {idx}: {plan['x']}")

        if new_columns == 0:
            #print(f"\nNo new columns added. Convergence reached at iteration {iteration}.")
            converged = True

    total_time = time.time() - start_time
    print(f"\nConverged in {iteration} iterations. Final ZDW = {final_obj:.2f}, Time = {total_time:.2f}s")

    # i = 1
    # for k,plan in enumerate(columns[i]):
    #     if z.get((i, k), 0) > 1e-4:
    #             selected = True
    #             for t in range(1, T+1):
    #                 print({
    #                     "Product": i,
    #                     "Period": t,
    #                     "Production": plan['x'][t]
    #                 })
    #             break

    FinalPlans = []
    for i in range(1, N+1):
        for k, plan in enumerate(columns[i]):
            if z[i, k] > 1e-4:
                FinalPlans.append([f"Product {i}", plan])
                break

    return FinalPlans, total_time, iteration, final_obj

In [ ]:
sheetNames = ['Data-20-12 (1)', 'Data-20-12 (2)', 'Data-20-24 (1)', 'Data-20-24 (2)', 'Data-100-24 (1)', 'Data-100-24 (2)', 'Data-200-24']
for sheet in sheetNames:
    print(f'Solving sheet: {sheet}')
    N, T, demand, c, h, f, r, kappa, tau, initialCost = defineData(sheet)
    finalPlans,  solveTime, iteration, final_obj = column_generation()
    print('----------')


## **5.Benders Decomposition**

In [ ]:
def Benders_decomposition(data, sheet_name, max_iterations):
    # Initialize master problem
    master = gp.Model("Benders Master Problem")
    
    T = data.T
    O = data.items


    start_time = time.time()
    time_limit = 30 * 60  # 30 minutes
    LBs = []
    UBs = []

    
    # Master problem variables
    y = master.addVars(O+1, T+1, vtype=GRB.BINARY, name="y")  # setup decisions
    theta = master.addVar(vtype=GRB.CONTINUOUS, lb=0, name="theta")  # variable for optimality cuts

    # Master problem objective
    master.setObjective(
        quicksum(quicksum(data.setup_cost[i, t] * y[i, t] for i in range(1, O+1)) for t in range(1, T+1)) +
        theta,
        GRB.MINIMIZE
        
    )
    # Production limit constraints contribution
    feas_cut_expr_prod = quicksum(
                quicksum( (sum(data.demand_forecast[i, q] for q in range(t, T+1)) * y[i, t])
                        for i in range(1, O+1)) #alright
                for t in range(1, T+1) 
        )

    # Flow constraints contribution
    feas_cut_expr_flow = quicksum( quicksum(data.demand_forecast[i,t] for i in range(1,O+1)) for t in range(1,T+1))

    master.addConstr(feas_cut_expr_flow<=feas_cut_expr_prod, name="initial cut")


    # Get initial y values from MILP solution
    y_vals = np.zeros((O+1,T+1))

    for i in range(1, O+1):
        y_vals[i, 1]=1


    UB=float("inf")
    LB=float("-inf")

    masterobj = []
    fesibilitysubobj = []
    
    for iteration in range(1, max_iterations):

        print(f"\nIteration {iteration}")
        
        # Get Farkas certificates from feasibility subproblem
        capacity_duals, prod_limit_duals, flow_duals, somma = solve_feasibility_subproblem(y_vals, data)
    
        if somma > 1e-6:  # If artificial variables are used, add feasibility cut
            #subproblem infeasible, need to add feasibility cut
            print("Subproblem is infeasible → Adding feasibility cut")
                
            # Capacity constraints contribution
            feas_cut_expr_cap = quicksum(
                +capacity_duals[t] * (data.capacity[t, 1] - quicksum(data.item_requirements[i, 2] * y[i, t]
                for i in range(1, O+1))) #alright
                for t in range(1, T+1)
            )

            # Production limit constraints contribution
            feas_cut_expr_prod = quicksum(
                quicksum(+prod_limit_duals[i, t] * 
                        (sum(data.demand_forecast[i, q] for q in range(t, T+1)) * y[i, t])
                        for i in range(1, O+1)) #alright
                for t in range(1, T+1)
            )

            # Flow constraints contribution
            feas_cut_expr_flow = quicksum( quicksum(+flow_duals[i, t]*data.demand_forecast[i,t] for i in range(1,O+1)) for t in range(1,T+1)
                    
            )
            # Add combined feasibility cut 
            master.addConstr(
                feas_cut_expr_cap + feas_cut_expr_prod + feas_cut_expr_flow<= 0,
                name=f"feasibility_cut_{iteration}"
            )
                
            print(f"Added feasibility cut in iteration {iteration}")

        else:
            print("Primal subproblem is optimal → Adding optimality cut")

            subprob, x_vals, s_vals = create_and_solve_subprob(y_vals, data)

            # Get dual values from optimal subproblem
            capacity_duals = {}
            prod_limit_duals = {}
            
            for t in range(1, T+1):
                constr = subprob.getConstrByName(f"capacity_{t}")
                capacity_duals[t] = constr.Pi
                
            for t in range(1, T+1):
                for i in range(1, O+1):
                    constr = subprob.getConstrByName(f"prod_limit_{i}_{t}")
                    prod_limit_duals[i,t] = constr.Pi

            # Add optimality cut
            opt_cut_expr = quicksum(
                +capacity_duals[t] * (data.capacity[t, 1] - quicksum(data.item_requirements[i, 2] * y[i, t]
                for i in range(1, O+1)))
                for t in range(1, T+1)
            ) + quicksum(
                quicksum(+prod_limit_duals[i, t] * 
                        (sum(data.demand_forecast[i, q] for q in range(t, T+1)) * y[i, t])
                        for i in range(1, O+1))
                for t in range(1, T+1)
            )
            + quicksum( quicksum(+flow_duals[i, t]*data.demand_forecast[i,t] for i in range(1,O+1)) for t in range(1,T+1))

            
            
            master.addConstr(theta >= opt_cut_expr, name=f"optimality_cut_{iteration}")
            print(f"Added optimality cut in iteration {iteration}")
            
            
        # Solve updated master problem

        remaining_time = max(0, time_limit - (time.time() - start_time))
        master.setParam("TimeLimit", remaining_time)
        master.optimize()

        
        if master.status == GRB.OPTIMAL:
            # Update y values
            for t in range(1, T+1):
                for i in range(1, O+1):
                    y_vals[i][t] = y[i, t].X

        elapsed_time = time.time() - start_time
        if elapsed_time >= time_limit:
            print("Time limit reached. Stopping early.")
            break

        LB=master.ObjVal
        LBs.append(LB)

        if somma<=1e-6 and subprob.Status == GRB.OPTIMAL:
            UB=subprob.ObjVal
        
        if UB-LB<1:
            print("Converged!!!!!")
            print(f"UB: {master.ObjVal}")
            break


    # After the loop
    plt.figure(figsize=(10, 6))
    plt.plot(LBs, label='Lower Bound (LB)', marker='o')
    plt.plot(UBs, label='Upper Bound (UB)', marker='x')
    plt.xlabel("Iteration", fontsize=14)
    plt.ylabel("Objective Value", fontsize=14)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"LB_UB_{sheet_name}.png")
    plt.show()


    
            
            
    return y_vals,

def create_and_solve_subprob(y_vals, data): #THIS IS P1, THE SUBPROBLEM
    #IT IS HIGHLY POSSBILE THAT IT IS UNFEASIBLE FOR THE Ys PROVIDED

    #THIS IS ALRIGHT

    T=data.T
    O=data.items

    #now we need to do the subproblem
    sub=gp.Model("Subproblem")

    s=sub.addVars(O+1, T+1, vtype=GRB.CONTINUOUS, lb=0, name="s") #amount of inv of item i in period t
    x=sub.addVars(O+1, T+1, vtype=GRB.CONTINUOUS, lb=0, name="x") #amount produced of item i in period t


    #this is P1
    sub.setObjective(
        quicksum(quicksum( data.production_cost[i, t] * x[i, t] for i in range(1, O+1) ) for t in range(1, T+1))+
        quicksum(quicksum( data.holding_cost[i, t] * s[i, t] for i in range(1, O+1) ) for t in range(1, T+1)),
    GRB.MINIMIZE
    )  

    #constraint 2
    for t in range(1, T+1):
        for i in range(1, O+1):
            sub.addConstr( s[i, t-1] + x[i, t] - s[i, t] == data.demand_forecast[i, t], 
                         name=f"flow_{i}_{t}")

    #constraint 3
    for t in range(1, T+1):
        sub.addConstr(quicksum(x[i, t] * data.item_requirements[i, 1] + data.item_requirements[i,2] * y_vals[i][t]   for i in range(1, O+1)) <= data.capacity[t, 1], 
             name=f"capacity_{t}")
            
    #constraint 4
    for t in range(1, T+1):
        for i in range(1, O+1):
            sub.addConstr(x[i, t]-(quicksum(data.demand_forecast[i, q] 
                                          for q in range(t, T+1))) * y_vals[i][t] <= 0, 
                         name=f"prod_limit_{i}_{t}")

    #constraint 7
    for i in range(1, O+1):
        sub.addConstr(s[i, 0] == 0, name=f"init_inv_{i}")
        sub.addConstr(s[i, T] == 0, name=f"fin_inv_{i}")

    
    sub.optimize()
        

    return sub, x, s


def solve_feasibility_subproblem(y_vals, data):
    T = data.T
    O = data.items

    feas = gp.Model("Feasibility_Subproblem")
    # Variables
    s = feas.addVars(O+1, T+1, vtype=GRB.CONTINUOUS, lb=0, name="s")
    x = feas.addVars(O+1, T+1, vtype=GRB.CONTINUOUS, lb=0, name="x")

    # Artificial variables for each constraint
    a_flow = feas.addVars(O+1, T+1, vtype=GRB.CONTINUOUS, lb=0, name="a_flow")
    a_cap = feas.addVars(T+1, vtype=GRB.CONTINUOUS, lb=0, name="a_cap")
    a_prod = feas.addVars(O+1, T+1, vtype=GRB.CONTINUOUS, lb=0, name="a_prod")
    a_init = feas.addVars(O+1, vtype=GRB.CONTINUOUS, lb=0, name="a_init")
    a_final = feas.addVars(O+1, vtype=GRB.CONTINUOUS, lb=0, name="a_final")
    
    # Objective: minimize sum of artificial variables
    feas.setObjective(
        quicksum(a_flow[i, t] for i in range(1, O+1) for t in range(1, T+1)) +
        quicksum(a_cap[t] for t in range(1, T+1)) +
        quicksum(a_prod[i, t] for i in range(1, O+1) for t in range(1, T+1)) +
        quicksum(a_init[i] + a_final[i] for i in range(1, O+1)),
        GRB.MINIMIZE
    )

    # Flow constraints with artificial variables
    for t in range(1, T+1):
        for i in range(1, O+1):
            feas.addConstr(
                s[i, t-1] + x[i, t] - s[i, t] + a_flow[i, t] == data.demand_forecast[i, t],
                name=f"flow_{i}_{t}"
            )

    # Capacity constraints
    for t in range(1, T+1):
        feas.addConstr(
            quicksum(x[i, t] * data.item_requirements[i, 1] + data.item_requirements[i, 2] * y_vals[i][t] for i in range(1, O+1)) +
            a_cap[t] <= data.capacity[t, 1],
            name=f"capacity_{t}"
        )

    # Production limit constraints
    for t in range(1, T+1):
        for i in range(1, O+1):
            rhs = sum(data.demand_forecast[i, q] for q in range(t, T+1)) * y_vals[i][t]
            feas.addConstr(
                x[i, t] - rhs + a_prod[i, t] <= 0,
                name=f"prod_limit_{i}_{t}"
            )

    # Inventory constraints
    for i in range(1, O+1):
        feas.addConstr(s[i, 0] + a_init[i] == 0, name=f"init_inv_{i}")
        feas.addConstr(s[i, T] + a_final[i] == 0, name=f"fin_inv_{i}")

    feas.optimize()

    print(f"Feasibility subproblem objective value: {feas.ObjVal}")



    # Get Farkas duals from relaxed constraints
    capacity_duals = {}
    prod_limit_duals = {}
    flow_duals = {}
    
    # Get duals for capacity constraints
    for t in range(1, T+1):
            constr = feas.getConstrByName(f"capacity_{t}")
            capacity_duals[t] = constr.Pi
            #print(f"capacity duals: {capacity_duals[t]}")

        
    # Get duals for production limit constraints
    for t in range(1, T+1):
        for i in range(1, O+1):
            constr = feas.getConstrByName(f"prod_limit_{i}_{t}")
            prod_limit_duals[i,t] = constr.Pi
            #print(f"prod limit duals: {prod_limit_duals[i, t]}")
            
            
    # Get duals for flow constraints
    for t in range(1, T+1):
        for i in range(1, O+1):
            constr = feas.getConstrByName(f"flow_{i}_{t}")
            flow_duals[i,t] = constr.Pi
            #print(f"flow duals: {flow_duals[i, t]}")
        
    # Calculate sum of artificial variables for detecting infeasibility
    flow_artificial=sum(sum(a_flow[i, t].X for i in range(1,O+1)) for t in range(1, T+1))
    #print(f"flow artificial variables sum: {flow_artificial}")
    capacity_artificial=sum(a_cap[t].X for t in range(1, T+1))
    #print(f"capacity artificial variables sum: {capacity_artificial}")
    prod_artificial=sum(sum(a_prod[i, t].X for i in range(1,O+1)) for t in range(1, T+1))
    #print(f"prod artificial variables sum: {prod_artificial}")
    

    summ = flow_artificial+capacity_artificial+prod_artificial 

    return capacity_duals, prod_limit_duals, flow_duals, summ
    



    

    



In [ ]:
for sheet_name in sheet_list:
    data=read_data(xls, sheet_name)
    Benders_decomposition(data, sheet_name, 200)